# Batch Normalization(TensorFlow版本)
透過將每一個batch的資料正規化來縮短模型訓練時間

In [2]:
# 增加訓練速度 (丟進激活函數之前先做Batch Normalization，簡稱BN層)
import numpy as np
import tensorflow as tf
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()

        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):

        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]
    
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        
        self.dense1 = tf.keras.layers.Dense(units=100)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.ac1 = tf.keras.layers.LeakyReLU(alpha=0.2)
        
        self.dense2 = tf.keras.layers.Dense(units=10)
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.ac2 = tf.keras.layers.LeakyReLU(alpha=0.2)

    def call(self, inputs):         # [batch_size, 28, 28, 1]
        x = self.flatten(inputs)    # [batch_size, 784]
        x = self.dense1(x)          # [batch_size, 100]
        x = self.bn1(x)
        x = self.ac1(x)
        
        x = self.dense2(x)          # [batch_size, 10]
        x = self.bn2(x)
        x = self.ac2(x)

        output = tf.nn.softmax(x)
        return output

num_epochs = 5
batch_size = 1000
learning_rate = 0.001

model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.trainable_variables))
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

batch 0: loss 2.395871
batch 1: loss 2.302521
batch 2: loss 2.223227
batch 3: loss 2.127664
batch 4: loss 2.082026
batch 5: loss 2.010781
batch 6: loss 1.953032
batch 7: loss 1.892682
batch 8: loss 1.830975
batch 9: loss 1.719758
batch 10: loss 1.677893
batch 11: loss 1.581999
batch 12: loss 1.517661
batch 13: loss 1.482351
batch 14: loss 1.403175
batch 15: loss 1.358136
batch 16: loss 1.222202
batch 17: loss 1.243408
batch 18: loss 1.194086
batch 19: loss 1.110593
batch 20: loss 1.021773
batch 21: loss 0.942443
batch 22: loss 0.911492
batch 23: loss 0.880218
batch 24: loss 0.804583
batch 25: loss 0.768039
batch 26: loss 0.785902
batch 27: loss 0.724358
batch 28: loss 0.699926
batch 29: loss 0.649268
batch 30: loss 0.610541
batch 31: loss 0.590848
batch 32: loss 0.602088
batch 33: loss 0.553420
batch 34: loss 0.570544
batch 35: loss 0.587716
batch 36: loss 0.547053
batch 37: loss 0.570801
batch 38: loss 0.541263
batch 39: loss 0.543303
batch 40: loss 0.511966
batch 41: loss 0.480443
ba